In [1]:
%matplotlib ipympl
import torch
import hydra, os, time
from typing import Any, Dict, List, Tuple, Type, Optional, Union, Sequence, Mapping
from fmod.base.util.dates import date_list
from fmod.pipeline.downscale import Downscaler
from fmod.base.util.config import configure, cfg, cfg_date, cfg2args, pp
import xarray as xa
from fmod.pipeline.rescale import DataLoader
from datetime import date
from fmod.plot.multiscale import mplplot

In [2]:
hydra.initialize(version_base=None, config_path="../config")
configure('merra2-sr')
reference_date = date(1990, 6, 1)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
if torch.cuda.is_available():
    torch.cuda.set_device(device.index)

 *** Configuration merra2-sr initialized *** 

  --------- Opening log file:  '/explore/nobackup/projects/ilab/data/FMF/cache/logs/merra2-sr.log' ---------  



In [3]:
data_loader = DataLoader()
lowres: xa.DataArray  = data_loader.get_channel_array( "low",  reference_date, interp_nan=True )
highres: xa.DataArray = data_loader.get_channel_array( "high", reference_date, interp_nan=True )

In [4]:
downscaler = Downscaler( model="sfno" )
results: Dict[str,xa.DataArray] = downscaler.process( lowres, highres )
mplplot( results, fsize=4.0 )

SFNO: signal.shape=torch.Size([91, 144]), source_shape=(91, 144), target_shape=(361, 576)
 ---->>>> forward: shape=torch.Size([91, 144]) <--> nlatlon=(91, 144)
 ....... out_shape=[361, 576, 2], wts_shape=torch.Size([576, 361, 91]), mmax=576


RuntimeError: einsum(): subscript m has size 576 for operand 1 which does not broadcast with previously seen size 73

In [ ]:
downscaler = Downscaler( model="interp", method="linear" )
results: Dict[str,xa.DataArray] = downscaler.process( lowres, highres )
mplplot( results, fsize=4.0 )